In [1]:
import pandas as pd
import requests
import numpy as np
import time
import folium

In [2]:
# Define origin and destination coordinates
origin = {"Latitude": 8.264014, "Longitude": 80.210510}  # Nochchiyagama
destination = {"Latitude": 8.538778, "Longitude": 80.492996}  # Medawachchiya

In [3]:
# Make a request to the OSRM API to get the route
response = requests.get(
    f"http://router.project-osrm.org/route/v1/car/{origin['Longitude']},{origin['Latitude']};{destination['Longitude']},{destination['Latitude']}?overview=full&geometries=geojson"
)

In [4]:
data = response.json()

In [5]:
data

{'code': 'Ok',
 'routes': [{'geometry': {'coordinates': [[80.210328, 8.263934],
     [80.210446, 8.26367],
     [80.210835, 8.26385],
     [80.212091, 8.264438],
     [80.215307, 8.265912],
     [80.216566, 8.266473],
     [80.221407, 8.268629],
     [80.221558, 8.268697],
     [80.223288, 8.269467],
     [80.223468, 8.269547],
     [80.224286, 8.269912],
     [80.226199, 8.270763],
     [80.227337, 8.27127],
     [80.227729, 8.271445],
     [80.229255, 8.272022],
     [80.229379, 8.272069],
     [80.230915, 8.27265],
     [80.231415, 8.272839],
     [80.231651, 8.272929],
     [80.233865, 8.273768],
     [80.235082, 8.27423],
     [80.235266, 8.274296],
     [80.238244, 8.275423],
     [80.240149, 8.276143],
     [80.242369, 8.277001],
     [80.245102, 8.278057],
     [80.246139, 8.278457],
     [80.248149, 8.279233],
     [80.248899, 8.2795],
     [80.250437, 8.280046],
     [80.250741, 8.280116],
     [80.252186, 8.280394],
     [80.252697, 8.280489],
     [80.255284, 8.281525],
   

In [6]:
route = data["routes"][0]["geometry"]["coordinates"]

In [7]:
 # Create a map centered on the origin
m = folium.Map(location=[origin["Latitude"], origin["Longitude"]], zoom_start=10)

In [8]:
# Add the road path to the map
folium.PolyLine(
            [(coord[1], coord[0]) for coord in route], color="blue", weight=5, opacity=0.7
        ).add_to(m)

In [9]:
# Add markers for the origin and destination
folium.Marker(
            [origin["Latitude"], origin["Longitude"]],
            popup="Origin: Nochchiyagama",
            icon=folium.Icon(color="green"),
        ).add_to(m)
folium.Marker(
            [destination["Latitude"], destination["Longitude"]],
            popup="Destination: Medawachchiya",
            icon=folium.Icon(color="red"),
        ).add_to(m)

In [10]:
m

In [11]:
df = pd.read_csv('../data/data_gps.csv')
df.head()

,NO,INVOICE NO,CUSTOMER NAME,ADDRESS,Invoice Value,Rep,LATITUDE,LONGITUDE
0,1,16529,Purasanda Trade Center,Nochchiyagama,21114.0,Dilshan,8.264014,80.210510
1,2,16437,Silumina Mega,Medawachchiya,240000.0,Dilshan,8.538778,80.492996
2,3,16539,Max City,Medirigiriya,48000.0,Dilshan,8.142325,80.971342
3,29,16560,Wenura Enterprises,Dambulla,154000.0,Dilshan,7.874203,80.651092
4,4,16528,Macro communication,Chilaw,48000.0,Dinushan,7.576507,79.795676


In [12]:
df.shape

(31, 8)

In [13]:
# Extract unique addresses and their corresponding latitude/longitude
if "ADDRESS" in df.columns and "LATITUDE" in df.columns and "LONGITUDE" in df.columns:
    unique_locations = df.drop_duplicates(subset=["ADDRESS"])[
        ["ADDRESS", "LATITUDE", "LONGITUDE"]
    ]
else:
    raise ValueError(
        "CSV file must contain 'ADDRESS', 'LATITUDE', and 'LONGITUDE' columns"
    )

In [14]:
unique_locations.shape

(28, 3)

In [15]:
adress = unique_locations['ADDRESS'].to_list()
adress[:5]

['Nochchiyagama', 'Medawachchiya', 'Medirigiriya', 'Dambulla', 'Chilaw']

In [16]:
locations = list(zip(unique_locations['LATITUDE'], unique_locations['LONGITUDE']))

In [17]:
# Number of locations
num_locations = len(locations)

In [18]:
# Initialize distance matrix with zeros
distance_matrix = np.zeros((num_locations, num_locations))

In [19]:
# OSRM API base URL
osrm_base_url = "http://router.project-osrm.org/route/v1/car"

In [20]:
# Function to get distance from OSRM API
def get_osrm_distance(origin, destination):
    """
    Get the distance between two coordinates using OSRM API.
    :param origin: (latitude, longitude)
    :param destination: (latitude, longitude)
    :return: Distance in meters
    """
    url = f"{osrm_base_url}/{origin[1]},{origin[0]};{destination[1]},{destination[0]}?overview=false"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if "routes" in data and len(data["routes"]) > 0:
            return data["routes"][0]["distance"] / 1000  # Convert meters to km
    return np.inf  # Return infinity if no valid route is found

In [21]:
# Build the distance matrix
for i in range(num_locations):
    for j in range(num_locations):
        if i < j:  # No need to calculate distance for the same locations
            dis = get_osrm_distance(locations[i], locations[j])
            distance_matrix[i, j] = dis
            distance_matrix[j, i] = dis
            time.sleep(0.2)  # To prevent rate limiting

In [22]:
distance_matrix[:4,:4]

array([[  0.    ,  53.5289, 117.1457,  76.1327],
       [ 53.5289,   0.    , 109.7074,  79.4945],
       [117.1457, 109.7074,   0.    ,  57.749 ],
       [ 76.1327,  79.4945,  57.749 ,   0.    ]])

In [23]:
# Create a DataFrame for better readability
distance_df = pd.DataFrame(distance_matrix, index=adress, columns=adress)

In [24]:
# distance_df.to_csv("../data/osrm_distance_matrix.csv")

In [1]:
# Display the distance matrix
import ace_tools as tools

tools.display_dataframe_to_user(name="OSRM Distance Matrix", dataframe=distance_df)

print("Distance matrix saved as 'osrm_distance_matrix.csv'.")

ModuleNotFoundError: No module named 'ace_tools'